In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import itertools
import re
import pickle

In [3]:
def remove_non_english(df):
    df = df[df['english'] == 1].copy()
    
    # doesn't contain 3 or more non-ascii characters
    df = df[~df['name'].str.contains('[^\u0001-\u007F]{3,}')]
    
    df = df.drop('english', axis=1)
    
    return df


def calc_rating(row):

    import math

    pos = row['positive_ratings']
    neg = row['negative_ratings']

    total_reviews = pos + neg

    average = pos / total_reviews

    score = average - (average*0.5) * 2**(-math.log10(total_reviews + 1))

    return score * 100


def get_unique(series):
    return set(list(itertools.chain(*series.apply(lambda x: [c for c in x.split(';')]))))


def process_cat_gen_tag(df):
    cat_cols = get_unique(df['categories'])
    
    # remove comments to use all categories
    cat_cols = [
#         'Local Multi-Player',
#         'MMO',
#         'Mods',
        'Multi-player',
#         'Online Co-op',
#         'Online Multi-Player',
        'Single-player'
    ]
    
    for col in sorted(cat_cols):
        col_name = re.sub(r'[\s\-\/]', '_', col.lower())
        col_name = re.sub(r'[()]', '', col_name)
        
        df[col_name] = df['categories'].apply(lambda x: 1 if col in x.split(';') else 0)
        
    gen_cols = get_unique(df['genres'])
    
    # remove comments to use all genres. Currently uses main ones as per steam store home page
    gen_cols = [
#         'Accounting',
        'Action',
        'Adventure',
#         'Animation & Modeling',
#         'Audio Production',
        'Casual',
#         'Design & Illustration',
#         'Documentary',
#         'Early Access',
#         'Education',
#         'Free to Play',
#         'Game Development',
#         'Gore',
        'Indie',
        'Massively Multiplayer',
#         'Nudity',
#         'Photo Editing',
        'RPG',
        'Racing',
#         'Sexual Content',
        'Simulation',
#         'Software Training',
        'Sports',
        'Strategy'
#         'Tutorial',
#         'Utilities',
#         'Video Production',
#         'Violent',
#         'Web Publishing'
    ]
    
    gen_col_names = []
    
    for col in sorted(gen_cols):
        col_name = col.lower().replace('&', 'and').replace(' ', '_')
        gen_col_names.append(col_name)
        
        df[col_name] = df['genres'].apply(lambda x: 1 if col in x.split(';') else 0)
    
    # remove "non-games" based on genre
    gen_sums = df[gen_col_names].sum(axis=1)
    df = df[gen_sums > 0]
        
    tag_cols = get_unique(df['steamspy_tags'])
    # df['top_tag'] = df['tags'].apply(lambda x: x.split(';')[0] if ';' in x else x)
    
    # display(cat_cols, gen_cols, tag_cols)
    df = df.drop(['categories', 'genres', 'steamspy_tags'], axis=1)
    
    return df

def process_dev_pub_tag(df):
    dev_cols = get_unique(df['developer'])
    dev_cols = [
        'Choice of Games',
        'Ripknot Systems',
        'Laush Dmitriy Sergeevich',
        'Nikita "Ghost_RUS"',
        'Dexion Games',
    ]
    for col in sorted(dev_cols):
        col_name = re.sub(r'[\s\-\/]', '_', col.lower())
        col_name = re.sub(r'[()]', '', col_name)
        df[col_name] = df['developer'].apply(lambda x: 1 if col in x.split(';') else 0)
    
    pub_cols = [
        'Big Fish Games',
        'Strategy First',
        'Ubisoft',
        'THQ Nordic',
        'Square Enix',
        'Sekai Project',
        'Choice of Games',
    ]
    
    for col in sorted(pub_cols):
        col_name = re.sub(r'[\s\-\/]', '_', col.lower())
        col_name = re.sub(r'[()]', '', col_name)
        
        df[col_name] = df['publisher'].apply(lambda x: 1 if col in x.split(';') else 0)
    df = df.drop(['developer', 'publisher'], axis=1)
    return df


def process_year(df):
    df['release_date'] = df['release_date'].astype('datetime64[ns]')
    df['release_year'] = df['release_date'].apply(lambda x: x.year)
    return df

    
def process_revenue(df):
    df['est_revenue'] = df['owners'] * df['price']
    return df


def process_price(df):
    cut_points = [-1, 0, 4, 10, 30, 50, 1000]
    label_names = ['free', 'very cheap', 'cheap', 'moderate', 'expensive', 'very expensive']
    
    df['price_categories'] = pd.cut(df['price'], cut_points, labels=label_names)
    
    return df


def pre_process(df):
    # english only
    df = remove_non_english(df)
    
    # windows only
    df = df[df['platforms'].str.contains('windows')].drop('platforms', axis=1).copy()
    
#     df['owners'] = df['owners'].str.split('-').apply(lambda x: (int(x[0]) + int(x[1])) / 2).astype(int)
    # df['owners'] = df['owners'].str.replace('[+,]', '').astype(int)
    
    # calculate ratings
    df['total_ratings'] = df['positive_ratings'] + df['negative_ratings']
    df['ratings_ratio'] = df['positive_ratings'] / df['total_ratings']
    df['weighted_rating'] = df.apply(calc_rating, axis=1)
    # df = df.drop(['positive', 'negative'], axis=1)
    
    df = df.rename(columns={'average_forever': 'average_playtime', 'median_forever': 'median_playtime'})
    
    df = process_year(df)
#     df = process_revenue(df)
    df = process_price(df)
    df = process_cat_gen_tag(df)
    df = process_dev_pub_tag(df)
    
    return df

raw1 = pd.read_csv('../data/steam.csv', delimiter=',')
df = pre_process(raw1)
df.dataframeName = 'steam'

df[['appid','positive_ratings','negative_ratings','total_ratings','ratings_ratio','weighted_rating']].head()
df.loc[df.owners == '5000000-10000000', 'owners'] = '>5000000'
df.loc[df.owners == '10000000-20000000', 'owners'] = '>5000000'
df.loc[df.owners == '20000000-50000000', 'owners'] = '>5000000'
df.loc[df.owners == '50000000-100000000', 'owners'] = '>5000000'
df.loc[df.owners == '100000000-200000000', 'owners'] = '>5000000'

df.drop('appid', axis=1, inplace=True)
df.drop('name', axis=1, inplace=True)
df.drop('release_date', axis=1, inplace=True)
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns after preprocessing')
df.head()

There are 25951 rows and 36 columns after preprocessing


,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,ratings_ratio,...,dexion_games,laush_dmitriy_sergeevich,"nikita_""ghost_rus""",ripknot_systems,big_fish_games,sekai_project,square_enix,strategy_first,thq_nordic,ubisoft
0,0,0,124534,3339,17612,317,>5000000,7.19,127873,0.973888,...,0,0,0,0,0,0,0,0,0,0
1,0,0,3318,633,277,62,>5000000,3.99,3951,0.839787,...,0,0,0,0,0,0,0,0,0,0
2,0,0,3416,398,187,34,>5000000,3.99,3814,0.895648,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1273,267,258,184,>5000000,3.99,1540,0.826623,...,0,0,0,0,0,0,0,0,0,0
4,0,0,5250,288,624,415,>5000000,3.99,5538,0.947996,...,0,0,0,0,0,0,0,0,0,0


In [16]:
fw = open('../data/dataset_{}_{}.pickle'.format(nRow,nCol),'wb')
# Pickle dictionary using protocol 0.
pickle.dump(df, fw)
fw.close()

In [17]:
fw = open('../data/dataset_{}_{}.pickle'.format(nRow,nCol),'rb')
df = pickle.load(fw)
df.head()

,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,total_ratings,ratings_ratio,...,dexion_games,laush_dmitriy_sergeevich,"nikita_""ghost_rus""",ripknot_systems,big_fish_games,sekai_project,square_enix,strategy_first,thq_nordic,ubisoft
0,0,0,124534,3339,17612,317,>5000000,7.19,127873,0.973888,...,0,0,0,0,0,0,0,0,0,0
1,0,0,3318,633,277,62,>5000000,3.99,3951,0.839787,...,0,0,0,0,0,0,0,0,0,0
2,0,0,3416,398,187,34,>5000000,3.99,3814,0.895648,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1273,267,258,184,>5000000,3.99,1540,0.826623,...,0,0,0,0,0,0,0,0,0,0
4,0,0,5250,288,624,415,>5000000,3.99,5538,0.947996,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def balance_dataset(df, col, ratio=0.5, balance_method='avg', random_state = 42):
    vc = df[col].value_counts()
    balanced_df = pd.DataFrame(columns=df.columns)
    if balance_method == 'avg':
        sample_size = int(df.shape[0] / len(vc))
        for i in vc.index:
            replace = (vc[i] < sample_size)
            temp = df[df[col] == i]
            balanced_df = balanced_df.append(temp.sample(n=sample_size, replace=replace, random_state=random_state), ignore_index=True)
        return balanced_df
    if balance_method == 'upward':
        highest_cat = vc.index[0]
        highest_num = vc[highest_cat]
        balanced_df = balanced_df.append(df[df[col] == highest_cat])
        for i in vc.index[1:]:
            num = vc[i]
            temp = df[df[col] == i]
            balanced_df = balanced_df.append(temp)
            sample_ratio = num / highest_num
            if sample_ratio < ratio:
                sample_size = int((ratio-sample_ratio) * highest_num)
                balanced_df = balanced_df.append(temp.sample(n=sample_size, replace=True, random_state=random_state), ignore_index=True)
        return balanced_df
    if balance_method == 'downward':
        lowest_cat = vc.index[-1]
        lowest_num = vc[lowest_cat]
        balanced_df = balanced_df.append(df[df[col] == lowest_cat])
        for i in vc.index[:-1]:
            num = vc[i]
            temp = df[df[col] == i]
            sample_ratio = lowest_num / num
            if sample_ratio < ratio:
                sample_size = int(sample_ratio * num)
                balanced_df = balanced_df.append(temp.sample(n=sample_size, replace=False, random_state=random_state), ignore_index=True)
        return balanced_df

In [8]:
target_variable = 'owners'
df = fw = open('../data/dataset_25951_36.pickle','rb')
df = pickle.load(fw)
method = 'upward'
df = balance_dataset(df, target_variable, balance_method=method)
print(df['owners'].value_counts())
nRow,nCol = df.shape
fw = open('../data/dataset_{}_{}_{}.pickle'.format(method, nRow,nCol),'wb')
pickle.dump(df, fw)
fw.close()

0-20000            17721
2000000-5000000     8860
1000000-2000000     8860
100000-200000       8860
500000-1000000      8860
50000-100000        8860
20000-50000         8860
>5000000            8860
200000-500000       8860
Name: owners, dtype: int64